# Scrap Text

In [ ]:
#@title import
%%capture
!pip install gnews newspaper3k
from gnews import GNews
from tqdm.notebook import tqdm_notebook as tqdm
from datetime import datetime, timedelta
import spacy
import re
import time
import pandas as pd
from google.colab import files

+ function to return news (dict)

In [ ]:
def get_related_news(google_news, keyword:str, start_date:datetime, end_date:datetime, bar=None)->list[dict]:
  google_news.start_date = (start_date.year, start_date.month, start_date.day)
  google_news.end_date = (end_date.year, end_date.month, end_date.day)

  if (bar is None):
    bar = tqdm(total=(end_date-start_date).days+1, desc="Getting News")
  # Get the news results
  results = google_news.get_news(keyword)
  num = len(results)

  if ((num >=99) and ((end_date - start_date)> timedelta(days=4))):
      # Recursively call the function for 1/2 the time and add them up
      mid_date = start_date + (end_date - start_date) / 2
      mid_date = datetime(mid_date.year, mid_date.month, mid_date.day)

      # Merge the results
      results = get_related_news(google_news, keyword, mid_date+timedelta(days=1), end_date, bar)\
              + get_related_news(google_news, keyword, start_date, mid_date, bar)

      #Check tqdm bar and close it
      if (bar.total == bar.n):
        bar.close()

      # Return direclty since results are already sorted
      return results

  sorted_results= sorted(results,
                         key=lambda x: datetime.strptime(x['published date'], "%a, %d %b %Y %H:%M:%S %Z"),
                         reverse=True)
  # Update tqdm bar
  update_proportion = (end_date - start_date).days+1
  bar.update(update_proportion)

  if (bar.total == bar.n):
    bar.close()

  time.sleep(10)

  return sorted_results

+ helper function for saving to csv, full dict and randoms

In [ ]:
def save_and_download_csv(df, keyword):
    df.to_csv(f"{keyword}.csv", index=False)
    files.download(f"{keyword}.csv")

In [ ]:
def save_200_sample(df,keyword):
    df_200 = df.sample(200)
    save_and_download_csv(df_200, f"{keyword}_200")

In [ ]:
def save_200_sample(df, keyword):
    # Filter the DataFrame to include only rows containing "COVID" or "coronavirus" in the "title" column
    filtered_df = df[df['title'].str.contains('COVID|coronavirus', case=False)]

    # Sample 200 rows from the filtered DataFrame
    df_200 = filtered_df.sample(200)

    # Save and download the sampled DataFrame as a CSV file
    save_and_download_csv(df_200, f"{keyword}_200")

+ scraaaaping

In [ ]:
# keyword = "COVID"
keyword = "symptom"

In [ ]:
start_date, end_date = datetime(2019, 10, 1 , 0, 0, 0), datetime(2023,10,1,0,0,0)

google_news = GNews()

results = get_related_news(google_news, keyword, start_date, end_date)
print(f"Num of news: {len(results)}")

Getting News:   0%|          | 0/1462 [00:00<?, ?it/s]

Num of news: 188


In [ ]:
# df = pd.DataFrame(results)
df = pd.read_csv(f"{keyword}.csv")

In [ ]:
# save_and_download_csv(df, keyword)
save_200_sample(df, keyword)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
covid = pd.read_csv(f"{keyword}.csv")

In [ ]:
covid.head()

,title,description,published date,url,publisher
0,"Multiple sclerosis (MS): Depression, anxiety m...","Multiple sclerosis (MS): Depression, anxiety m...","Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.medicalnewstoday.com', '..."
1,Surprising Menopause Symptoms - Everyday Health,Surprising Menopause Symptoms Everyday Health,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiX2h0d...,"{'href': 'https://www.everydayhealth.com', 'ti..."
2,Gluten-Free Diet Improves Fibromyalgia Symptom...,Gluten-Free Diet Improves Fibromyalgia Symptom...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTGh0d...,"{'href': 'https://www.hcplive.com', 'title': '..."
3,Menopausal Women With Obesity Endure Worse Sym...,Menopausal Women With Obesity Endure Worse Sym...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiOGh0d...,"{'href': 'https://www.medpagetoday.com', 'titl..."
4,7 Things Your Hair Reveals About Your Health -...,7 Things Your Hair Reveals About Your Health ...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiZGh0d...,"{'href': 'https://www.everydayhealth.com', 'ti..."


In [ ]:
# keyword = "COVID"
keyword = "symptom"

In [ ]:
to_process = pd.read_csv(f"{keyword}_200.csv")

In [ ]:
len(to_process)

200

In [ ]:
to_process.head()

,title,description,published date,url,publisher
0,COVID symptoms: 40% of coronavirus cases are a...,COVID symptoms: 40% of coronavirus cases are a...,"Tue, 14 Dec 2021 08:00:00 GMT",https://news.google.com/rss/articles/CBMifmh0d...,"{'href': 'https://www.usatoday.com', 'title': ..."
1,Coronavirus: Purple 'covid toes' symptom can l...,Coronavirus: Purple 'covid toes' symptom can l...,"Thu, 29 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiYGh0d...,"{'href': 'https://www.dailymail.co.uk', 'title..."
2,"Jill Biden tests positive for covid, with 'mil...","Jill Biden tests positive for covid, with 'mil...","Mon, 04 Sep 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://www.washingtonpost.com', 'ti..."
3,Is it Covid or Lyme disease? Key symptom to sp...,Is it Covid or Lyme disease? Key symptom to sp...,"Sun, 14 Aug 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTGh0d...,"{'href': 'https://www.mirror.co.uk', 'title': ..."
4,Coronavirus symptoms: Signs COVID-19 has impac...,Coronavirus symptoms: Signs COVID-19 has impac...,"Tue, 09 Mar 2021 08:00:00 GMT",https://news.google.com/rss/articles/CBMioQFod...,{'href': 'https://timesofindia.indiatimes.com'...


In [ ]:
google_news = GNews()

json_resp = to_process.to_dict(orient='records')
all_texts = []
for item in json_resp:
    time.sleep(1.5)
    article = google_news.get_full_article(item['url'])
    try:
        text = article.text
    except AttributeError:
        text = ""  # Append an empty string if article is None
    all_texts.append(text)

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://edmontonjournal.com/news/local-news/groundwork/hinshaw-cuts-runny-nose-from-list-of-covid-19-symptoms-forcing-isolation-groundwork on URL https://news.google.com/rss/articles/CBMiigFodHRwczovL2VkbW9udG9uam91cm5hbC5jb20vbmV3cy9sb2NhbC1uZXdzL2dyb3VuZHdvcmsvaGluc2hhdy1jdXRzLXJ1bm55LW5vc2UtZnJvbS1saXN0LW9mLWNvdmlkLTE5LXN5bXB0b21zLWZvcmNpbmctaXNvbGF0aW9uLWdyb3VuZHdvcmvSAQA?oc=5&hl=en-US&gl=US&ceid=US:en
An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.abc.net.au/news/2023-02-20/could-exercise-help-some-long-covid-sufferers/101975206 on URL https://news.google.com/rss/articles/CBMiXmh0dHBzOi8vd3d3LmFiYy5uZXQuYXUvbmV3cy8yMDIzLTAyLTIwL2NvdWxkLWV4ZXJjaXNlLWhlbHAtc29tZS1sb25nLWNvdmlkLXN1ZmZlcmVycy8xMDE5NzUyMDbSAShodHRwczovL2FtcC5hYmMubmV0LmF1L2FydGljbGUvMTAxOTc1MjA2?oc=5&hl=en-US&gl=US&ceid=US:en

In [ ]:
print(len(all_texts))

200


In [ ]:
df = pd.DataFrame(json_resp)
df_text = pd.DataFrame({'texts': all_texts})
df = pd.concat([df, df_text], axis=1)
df.head()

,title,description,published date,url,publisher,texts
0,COVID symptoms: 40% of coronavirus cases are a...,COVID symptoms: 40% of coronavirus cases are a...,"Tue, 14 Dec 2021 08:00:00 GMT",https://news.google.com/rss/articles/CBMifmh0d...,"{'href': 'https://www.usatoday.com', 'title': ...",Health officials have long wondered how many p...
1,Coronavirus: Purple 'covid toes' symptom can l...,Coronavirus: Purple 'covid toes' symptom can l...,"Thu, 29 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiYGh0d...,"{'href': 'https://www.dailymail.co.uk', 'title...",'Covid toes' that turn purple from the disease...
2,"Jill Biden tests positive for covid, with 'mil...","Jill Biden tests positive for covid, with 'mil...","Mon, 04 Sep 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://www.washingtonpost.com', 'ti...",First lady Jill Biden tested positive for the ...
3,Is it Covid or Lyme disease? Key symptom to sp...,Is it Covid or Lyme disease? Key symptom to sp...,"Sun, 14 Aug 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiTGh0d...,"{'href': 'https://www.mirror.co.uk', 'title': ...",Coronavirus: Key symptoms of the new Centaurus...
4,Coronavirus symptoms: Signs COVID-19 has impac...,Coronavirus symptoms: Signs COVID-19 has impac...,"Tue, 09 Mar 2021 08:00:00 GMT",https://news.google.com/rss/articles/CBMioQFod...,{'href': 'https://timesofindia.indiatimes.com'...,Respiratory symptoms are the classic marker of...


In [ ]:
df = df[["published date","texts"]]

In [ ]:
df["texts"][99]

'Headache is said to be one of the most common neurological symptoms of COVID-19 that could linger on even after all the other symptoms have disappeared.\n\n\n\n\n\nA 2020 study found that out of 130 participants positive with COVID-19, 74.6 percent had mild to moderate headaches. One-quarter of the patients had severe migraine-like headaches and it was noted that it was one of the earliest symptoms to appear.\n\n\n\n\n\nAnother study published in PubMed Central found that people who had COVID headaches developed tension headaches. Some of the characteristics include:\n\n\n\n\n\n- Moderate to severe pain\n\n- Pain on both sides of the head\n\n- Pain around the forehead including sides of the head and around the eyes\n\n- Tightness or pulsing sensation\n\n- Little response to OTC medications\n\n\n\n\n\nThe study also notes that a quarter of the participants experience headaches resembling a migraine. Symptoms include throbbing pain usually on one side of the head along with nausea and v

+ split sentence tokenization

In [ ]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")

def split_sentences(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

sentences = []
for text in df['texts']:
    if text.strip():  # Check if the text is not empty
        sentences.extend(split_sentences(text))

print(sentences[:5])

['Health officials have long wondered how many people who get COVID-19 show no symptoms.', "Because those who don't feel sick rarely get tested, experts have only been able to estimate the number of such asymptomatic cases.", 'But new research now shows more than 40% of those who tested positive for COVID-19 were asymptomatic, according to the study published Tuesday in JAMA Network Open.', 'Researchers looked at 95 studies from January 2020 to February 2021 consisting of nearly 30 million people in Asia, Europe, North America, South America and Africa.', 'More than 60% of confirmed COVID-19 cases among people under 20 were asymptomatic; nearly 50% in people 20 to 39; about 32% in people 40 to 59, and about 33% in those over 60.']


+ Write in text file

In [ ]:
with open(f'/content/textLine_{keyword}.txt', 'w') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [ ]:
def remove_empty_lines(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Filter out empty lines
    filtered_lines = [line for line in lines if line.strip()]

    # Write the modified lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(filtered_lines)

In [ ]:
remove_empty_lines(f"textLine_{keyword}.txt")

+ Number of tokens

In [ ]:
def count_words(filename):
    try:
        with open(filename, 'r') as file:
            content = file.read()
            words = content.split()
            return len(words)
    except FileNotFoundError:
        print("File not found.")
        return 0

filename = f'textLine_{keyword}.txt'
word_count = count_words(filename)
print(f"The file '{filename}' contains {word_count} words.")

The file 'textLine_symptom.txt' contains 117150 words.


In [ ]:
from google.colab import files

filename = f"textLine_{keyword}.txt"

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !wc -l textLine_COVID.txt
!wc -l textLine_symptom.txt

6442 textLine_symptom.txt
